In [12]:
import numpy as np
from mp_api.client import MPRester
from pymatgen.phonon.plotter import PhononBSPlotter
from lmapr1492 import get_plot_bs
from math import pi
from matplotlib import pyplot as plt
from lmapr1492 import plot_brillouin_zone, get_plot_bs, get_plot_dos, get_plot_bs_and_dos, get_branch_wavevectors, latex_fix

In [13]:
mp_key = "ER5V8i0JUw7k31Nk5IpkvogP61BDz9cw"
mp_id = "mp-569677"

In [14]:
with MPRester(mp_key) as m:
    ph_bs = m.get_phonon_bandstructure_by_material_id(mp_id)

Retrieving PhononBSDOSDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

**Phonon Bandstructure**

In [15]:
fig_ph_bs = get_plot_bs(ph_bs)
fig_ph_bs.update_yaxes(rangemode="tozero")
fig_ph_bs.show()

In [25]:
# Extraction des données de la structure phononique
ph_bs_plotter = PhononBSPlotter(ph_bs)
data = ph_bs_plotter.bs_plot_data()

In [26]:
distances = data["distances"]
Frequencies = data["frequency"]

directions = ["Γ - X", "Γ - L", "Γ - Z"]

In [31]:
# Identification des indices corrects des segments
labels = data["ticks"]["label"]
index_Gamma = next((i for i, lbl in enumerate(labels) if "\\Gamma" in lbl), None)
index_L = labels.index("L")  # Premier L
index_Z = [i for i, lbl in enumerate(labels) if lbl == "Z"][0]  # Premier Z
index_XQ = [i for i, lbl in enumerate(labels) if "X|Q" in lbl][0]  # X|Q

In [32]:
# Correspondance avec distances
zones = [index_Gamma, index_L, index_Z]
directions = ["Γ - X|Q", "Γ - L", "Γ - Z"]

In [33]:
a_values = np.zeros((3, 1000))
d_values = np.zeros((3, 1000))

fact = 100  # Conversion : Angstrom -> m, THz -> Hz

In [34]:
for i, zone in enumerate(zones):
    q = distances[zone]
    F = Frequencies[zone]
    
    # Sélection de la branche acoustique
    for j, branch in enumerate(F):
        if branch[0] == 0 or branch[-1] == 0:
            freq = branch
            break
    
    # Ajustement linéaire
    x = np.array([q[0], q[1]])
    y = np.array([freq[0], freq[1]])
    Coefficients = np.polyfit(x, y, 1)
    
    # Génération des points de la droite ajustée
    a_values[i] = np.linspace(q[0], q[-1], 1000)
    d_values[i] = Coefficients[0] * a_values[i] + Coefficients[1]
    
    print("La vitesse du son de la branche acoustique n°", branche[i]+1,
          "dans la direction",directions[i],"=", 2*pi*abs(Coefficients[0])*fact, "[m/s].")

La vitesse du son de la branche acoustique n° 1 dans la direction Γ - X|Q = 2191.3394283205957 [m/s].
La vitesse du son de la branche acoustique n° 2 dans la direction Γ - L = 2280.9084009090957 [m/s].
La vitesse du son de la branche acoustique n° 3 dans la direction Γ - Z = 2051.678519774708 [m/s].


**Zone de brillouin**

In [22]:
plot_brillouin_zone(ph_bs.structure)